In [2]:
# Import Python package
import queue
from threading import Event

from supernovacontroller.sequential import SupernovaDevice
from supernovacontroller.sequential.i3c import SupernovaI3CBlockingInterface

## Getting started

#### 1. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova Device class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
device = SupernovaDevice()

#### 2. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(usb_address : str)
```
- ``usb_address: str``: The OS HID path assigned to the device. This can be obtained using the ``device.getAllConnectedSupernovaDevices()`` method.

In [4]:
# Use the method by default to connect to only one connected Supernova device.
device.open()

{'hw_version': 'C',
 'fw_version': '2.4.0',
 'serial_number': '0A071E8504C72D59A5FFE0350EDD2A9E',
 'manufacturer': 'Binho LLC',
 'product_name': 'Binho Supernova'}

#### 3. Define and register a callback to handle responses and notifications from Supernova

To handle notifications from Supernova, a callback function must be defined and registered with a filter to identify the notification. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def on_notification(self, name : str, filter_func : callable, handler_func : callable) -> None:``

In [5]:
caught_ibis = queue.SimpleQueue()
caught_hot_join = Event()
pic_hot_joined_info = {}

# IBI configuration
def is_ibi(name, message):
    return message['name'].strip() == "I3C IBI NOTIFICATION" and message['header']['type'] == "IBI_NORMAL" and message['MDB']['value'] != 2

def handle_ibi(name, message):
    global caught_ibis
    ibi_info = {'dynamic_address': message['header']['address'],  'controller_response': message['header']['response'], 'mdb':message['payload'][0], 'payload':message['payload'][1:]}
    caught_ibis.put(ibi_info)

device.on_notification(name="ibi", filter_func=is_ibi, handler_func=handle_ibi)

def is_hot_join_ibi(name, message):
    return message['name'].strip() == "I3C IBI NOTIFICATION" and message['header']['type'] == "IBI_HOT_JOIN"

def handle_hot_join_ibi(name, message):
    global caught_hot_join
    global pic_hot_joined_info
    pic_hot_joined_info = {'dynamic_address': message['header']['address'], 'bcr': message['bcr'], 'dcr': message['dcr'], 'pid': message['pid']}
    caught_hot_join.set()

device.on_notification(name="hot_join", filter_func=is_hot_join_ibi, handler_func=handle_hot_join_ibi)

#### 4. Initialize the I3C interface

To operate I3C over the Supernova we must initialize the interface.

In [6]:
i3c : SupernovaI3CBlockingInterface = device.create_interface("i3c.controller")

## I3C Protocol API

### 1. Configuring the supernova/PIC18QF16Q20/I2C FRAM Setup

To facilitate the Protocol Translator test:

**Necessary Firmware:**
- Supernova firmware version 2.0.0
- Supernova SDK version 2.0.0
- Custom image for the PIC18QF16Q20 protocol translator (located in the application folder)

**Connection Arrangement:**

<img src="assets/block_diagram.png" alt="connection" width="60%">

<img src="assets/connection.png" alt="connection_diagram" width="40%">

- Link the supernova High voltage I3C tiger eye to Qwiic connector with the I3C Qwiic connector on the Supernova PIC18QF16Q20 board.
- Establish connections for SDA and SCL from the I2C/I3C Qwiic connector on the Supernova PIC18QF16Q20 board to the corresponding SCL/SDA channels of the I2C FRAM.
- Connect VCC and GND from the I2C Qwiic on the Supernova to the VCC and GND of the I2C FRAM.

   ![Connection Image Placeholder]

**Testing Procedure:**

The objective of this test is to utilize the PIC18QF16Q20 as a protocol translator, issuing custom I3C commands to execute the following transactions:
- I2C write


<img src="assets/i2c_write.png" alt="connection" width="80%">

- I2C read

![I2C_read](assets/i2c_read.png)

- SPI write

![SPI_write](assets/spi_write.png)

- SPI read

![SPI_read](assets/spi_read.png)

- SPI write followed by read

<img src="assets/spi_write_read.png" alt="connection" width="70%">


For I2C transactions, an I2C FRAM will be employed for data read and write operations. To assess SPI functionality, the Supernova PIC18QF16Q20 board integrates a W25Q64JV (64M-bit) SPI Serial Flash, allowing for the execution of commands and read/write operations.

### 2. Initialize the Supernova I3C peripheral as controller 

We set up the bus parameters Push Pull and Open Drain Frequencies aswell as the voltage to be utilized.  
Once configured we can initialize the bus and with it, initialize the Supernova as a controller.

In [7]:
i3c.set_parameters(i3c.I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ, i3c.I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ)
i3c.set_bus_voltage(1200)
i3c.init_bus()

(False, {'errors': ['NACK_ERROR']})

### 3. Hot Joining the PIC18QF16Q20

Once the I3C Bus has been set up, we can hot-join the PIC18QF16Q20 to start using it. 

When ready, run the following cell and whilst it is runnning connect the PIC18QF16Q20

In [8]:
# Initialize and scan the I3C bus.
print("Awaiting PIC Hot Join")
successful_hotjoin = caught_hot_join.wait(15)

if not successful_hotjoin:
    print("Could not hot join PIC, timeout exceeded")
    reading_ibi = "NOTHING"

pic_dynamic_address = int(pic_hot_joined_info['dynamic_address'])

print(i3c.targets())

Awaiting PIC Hot Join
(True, [{'static_address': 0, 'dynamic_address': 8, 'bcr': 30, 'dcr': 198, 'pid': ['0x06', '0x9a', '0x00', '0x00', '0x00', '0x00']}])


### 4. Communicate with the I2C peripheral

To verify the proper functionality of the I2C FRAM, we will execute the following transactions:

1. **I2C Write Transaction:**
   - Write data [0x06,0x07,0x08,0x09,0x0A] to the register [0x00,0x00].

2. **I2C Write to Specific Register:**
   - Direct an I2C write to the register [0x00,0x00].

3. **I2C Read Transaction:**
   - Perform an I2C read of length 5 to retrieve the previously written data.

For this we will define the registers and data to be utilized

In [9]:
ADDRESS_TO_WRITE_READ = 0x50
SHIFTED_ADDRESS = ADDRESS_TO_WRITE_READ << 1
REGISTER_TO_WRITE_READ = [0x00, 0x00]
DATA_TO_WRITE = [0x06, 0x07, 0x08, 0x09, 0x0A]
BYTES_TO_READ = 0x05

PIC_I2C_WRITE_COMMAND = 0x40
PIC_I2C_READ_COMMAND = 0x20

> Note that the target we will be utilizing has the address `0x50`, but to use it through the PIC18QF16Q20 we must shift it by 1 bit 

#### Execution Steps:

To write to I2C we must execute an I3C write to the dynamic address of the PIC18QF16Q20 (0x08). 
In the data to be written we will include:
   - Command ID for the I2C write command (0x40)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Register to write to [0x00, 0x00]
   - Data to be written [0x06,0x07,0x08,0x09,0x0A]
All these data we defined previously, so the write command is as simple as folows:

In [10]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_I2C_WRITE_COMMAND, SHIFTED_ADDRESS] + REGISTER_TO_WRITE_READ + DATA_TO_WRITE)
print("Write at Register 00 00 data 06 07 08 09 0A :", "SUCCESS" if success else "FAIL")

Write at Register 00 00 data 06 07 08 09 0A : SUCCESS


For the second transaction, resetting the data pointer to the [0x00, 0x00] register is achieved by performing a write with the following data:
   - Command ID for the I2C write command (0x40)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Register to write to [0x00,0x00]
   - Empty data payload

In [11]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_I2C_WRITE_COMMAND, SHIFTED_ADDRESS] + REGISTER_TO_WRITE_READ)
print("Reset data pointer to 00 00", "SUCCESS" if success else "FAIL")

Reset data pointer to 00 00 SUCCESS


For the final transaction, executing a read command involves sending the following I3C data:

   - Command ID for the I2C read command (0x20)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Amount of data to read (0x05)

This initiates an I2C read to the designated address with the specified length. The data resulting from this read will be received via I3C in an IBI with payload. This IBI includes a mandatory data byte of 0x00, and the remaining payload constituting the read data.

With how we have configured the IBIs previously, we can just wait until we get one and see its contents.

In [12]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_I2C_READ_COMMAND, SHIFTED_ADDRESS, BYTES_TO_READ])
print("Read 5 bytes", "SUCCESS" if success else "FAIL")

try:
    reading_ibi = caught_ibis.get(timeout=3)
except queue.Empty:
    print("Did not catch any IBI readings, is everything properly connected?")
    reading_ibi = "NOTHING"

print("Read from I2C target:", reading_ibi)

Read 5 bytes SUCCESS
Read from I2C target: {'dynamic_address': 8, 'controller_response': 'IBI_ACKED_WITH_PAYLOAD', 'mdb': 0, 'payload': [6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## 5. Communicate with the SPI peripheral

To validate the SPI communication, we will execute specific SPI commands tailored for the Supernova PIC18QF16Q20 board's SPI FLASH. Here are some examples:

1. **Get JEDEC ID:**
   - Execute the command to retrieve the JEDEC ID from the SPI FLASH.

2. **Write Status Register-1:**
   - Perform the command to write to Status Register-1.

3. **Read Status Register-1:**
   - Execute the command to read from Status Register-1.

These commands will allow us to assess the SPI communication and validate the Supernova PIC18QF16Q20 board's interaction with the SPI FLASH.  
For this we will define the following constants

In [13]:

SPI_WRITE_ENABLE_COMMAND = 0x06
SPI_READ_STATUS_REGISTER1_COMMAND = 0x05
SPI_GET_JEDEC_COMMAND = 0x9F
SPI_STATUS_REGISTER1 = 0x01

DATA_TO_WRITE = 0x60
BYTES_TO_READ = 0x03

PIC_SPI_WRITE_READ_COMMAND = 0x60
PIC_SPI_READ_COMMAND = 0x21
PIC_SPI_WRITE_COMMAND = 0x41

To obtain the JEDEC ID, the SPI command 0x9F should be issued via SPI, followed by reading 3 bytes of data. To accomplish this, the write/read SPI command (0x60) for the PIC18QF16Q20 should be utilized, requiring the following data to be sent via I3C:

- Command ID for the SPI write/read command (0x60)
- Amount of data to read (0x03)
- Data to write (the SPI command for getting JEDEC ID 0x9F)

Similar to the I2C read transaction, once the data is retrieved via SPI, an IBI will be sent via I3C with the mandatory data byte (MDB) set to 0x00 and the 3 bytes of read data.

In [14]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_SPI_WRITE_READ_COMMAND, BYTES_TO_READ, SPI_GET_JEDEC_COMMAND])
print("Read JEDEC ID:", "SUCCESS" if success else "FAIL")

try:
    reading_ibi = caught_ibis.get(timeout=3)
except queue.Empty:
    print("Did not catch any new IBI readings, is everything properly connected?")
    exit(1)

print("Read from SPI target:", reading_ibi)


Read JEDEC ID: SUCCESS
Read from SPI target: {'dynamic_address': 8, 'controller_response': 'IBI_ACKED_WITH_PAYLOAD', 'mdb': 0, 'payload': [239, 64, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


To perform the Write Status Register-1 SPI command, we need to first enable write transactions by issuing the Write Enable command (0x06). The necessary data to be sent via I3C includes:

- Command ID for the SPI write command (0x41)
- Data to write (the SPI Write Enable command value 0x06)

In [15]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_SPI_WRITE_COMMAND, SPI_WRITE_ENABLE_COMMAND])
print("Enable writing in the Memory via Write Enable", "SUCCESS" if success else "FAIL")

Enable writing in the Memory via Write Enable SUCCESS


To execute the Write Status Register-1 SPI command, the requisite data to be sent via I3C includes:

- Command ID for the SPI write command (0x41)
- Data to write
    - Write Status Register-1 SPI command (0x01)
    - Data to be written to the register (0x60)

Configuring the Register-1 with the value 0x60 involves setting:



![I2C_read](assets/Register.png)

- SEC to 1: Block Protect Bits (BP2, BP1, BP0) protect 4KB Sectors.
- TB to 1: Block Protect Bits (BP2, BP1, BP0) protect from the Bottom.






In [16]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_SPI_WRITE_COMMAND, SPI_STATUS_REGISTER1, DATA_TO_WRITE])
print("Write in SPI Status Register data", "SUCCESS" if success else "FAIL")

Write in SPI Status Register data SUCCESS


To execute the Read Status Register-1 SPI command, the following I3C data must be transmitted:

- Command ID for the SPI write/read command (0x60)
- Amount of data to read (0x01 from the register) 
- Data to write (SPI command for reading the Status Register-1, 0x05)

Similar to the previous transactions, the data from the Status Register-1 value is transmitted via IBI on the I3C bus. As configured earlier, the expected value should be 0x60.

In [17]:
(success, _) = i3c.write(pic_dynamic_address, i3c.TransferMode.I3C_SDR, [], [PIC_SPI_WRITE_READ_COMMAND, SPI_STATUS_REGISTER1, SPI_READ_STATUS_REGISTER1_COMMAND])
print("Read in SPI Memory", "SUCCESS" if success else "FAIL")

try:
    reading_ibi = caught_ibis.get(timeout=3)
except queue.Empty:
    print("Did not catch any new IBI readings, is everything properly connected?")
    reading_ibi = "NOTHING"

print("Read from SPI target:", reading_ibi)

Read in SPI Memory SUCCESS
Read from SPI target: {'dynamic_address': 8, 'controller_response': 'IBI_ACKED_WITH_PAYLOAD', 'mdb': 0, 'payload': [96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
